In [ ]:
import evotorch
from evotorch.logging import StdOutLogger
from evotorch.algorithms import GeneticAlgorithm
from evotorch.operators import TwoPointCrossOver, PolynomialMutation

import IPython.display as ipd
import torch

from ohc.vst import VSTHost
from ohc.fitness.fitness import FitnessFunction
from ohc.fitness.clap import CLAPSimilarity

%load_ext autoreload
%autoreload 2

In [ ]:
PLUGIN_PATH = "/home/jshier/development/one-hand-clapping/dexed-0.9.6-lnx/Dexed.vst3"
SAMPLE_RATE = 48000
BLOCK_SIZE = 512
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
vst = VSTHost(PLUGIN_PATH, "fixed", SAMPLE_RATE, BLOCK_SIZE)

In [ ]:
params = vst.list_params(filter_midi_cc=True)
active_params = [p for p in params if p != "Bypass"]
vst.set_active_params(active_params)

In [ ]:
clap = CLAPSimilarity(sample_rate=SAMPLE_RATE)

In [ ]:
text_target = ["dog bark"]
fitness = FitnessFunction(vst, clap, text_target=text_target)

In [ ]:
problem = evotorch.Problem(
    ["max"] * len(text_target),
    fitness.compute,
    solution_length=len(active_params),
    initial_bounds=(0, 1),
    bounds=(0, 1),
)

In [ ]:
ga = GeneticAlgorithm(
    problem,
    popsize=100,
    operators=[
        TwoPointCrossOver(problem, tournament_size=4),
        PolynomialMutation(problem),
    ],
    re_evaluate=False,
)

In [ ]:
_ = StdOutLogger(ga)  # Report the evolution's progress to standard output
ga.run(1000)  # Run the algorithm for 100 generations
print("Solution with best fitness ever:", ga.status["best"])

In [ ]:
ga.status["best"].shape

In [ ]:
best_discovered_solution = ga.status['best']

In [ ]:
audio, _ = vst.render_now(best_discovered_solution.values.unsqueeze(0), 100, 1.0, 1.0)[0]

In [ ]:
ipd.Audio(audio, rate=SAMPLE_RATE)